In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import t

In [2]:
np.random.seed(11)
ticker = "TRP.RO"
start_date = "2010-01-01"
confidence = 0.95
num_simulations = 50_000
horizon_days = 1

In [3]:
try:
    stock = yf.Ticker(ticker)
    df = stock.history(period="max")
except Exception as e:
    print("Error downloading", e)
    df = None
if df is None or df.empty:
    raise ValueError(f"No data returned for {ticker}")
if "Adj Close" in df.columns:
    df = df[["Adj Close"]].rename(columns={"Adj Close": "stock"})
else:
    df = df[["Close"]].rename(columns={"Close": "stock"})

In [4]:
#Use adjusted prices for VaR
prices = df["stock"].dropna()
returns = prices.pct_change().dropna()
current_price = float(prices.iloc[-1])
r = returns.values

In [5]:
#VaR Historical
sampled = np.random.choice(r, size=(num_simulations, horizon_days), replace=True)
cumulative_return = np.prod(1 + sampled, axis=1) - 1
simulated_prices = current_price * (1 + cumulative_return)
pnl = simulated_prices - current_price
hb_var = -np.percentile(pnl, (1 - confidence) * 100)

In [6]:
#Student-t Monte Carlo VaR
dff, loc, scale = t.fit(r)
simulated_t = t.rvs(df=dff,loc=0,scale=scale,size=(num_simulations, horizon_days))
cumulative_return_t = np.prod(1 + simulated_t, axis=1) - 1
simulated_prices_t = current_price * (1 + cumulative_return_t)
pnl_t = simulated_prices_t - current_price
t_var = -np.percentile(pnl_t, (1 - confidence) * 100)

In [7]:
print(f"Last price: {current_price:.4f}")
print(f"Historical Bootstrap VaR (95%): {hb_var:.4f} RON")
print(f"On 95% of days, you should expect not to lose more than {(hb_var/current_price)*100:.4f}% in a day")
print(f"Student-t Monte Carlo VaR (95%): {t_var:.4f} RON")
print(f"On 95% of days, you should expect not to lose more than {(t_var/current_price)*100:.4f}% in a day")

Last price: 0.4430
Historical Bootstrap VaR (95%): 0.0185 RON
On 95% of days, you should expect not to lose more than 4.1680% in a day
Student-t Monte Carlo VaR (95%): 0.0198 RON
On 95% of days, you should expect not to lose more than 4.4689% in a day
